In [ ]:
import numpy as np
import yaml
import matplotlib.pyplot as plt

#Import hardware classes
from pyRTC.hardware.OOPAOInterface import OOPAOInterface
from pyRTC.SlopesProcess import SlopesProcess
from pyRTC.Pipeline import *
from pyRTC.Loop import Loop
from pyRTC.hardware.LoopGymInterface import LoopGymInterface

import gymnasium
from rtgym import DEFAULT_CONFIG_DICT
import time

In [ ]:
"""
Shared memory in python is a bit annoying, we are required to unlink it from the garbage collector
so that it will stick around in between runs, however sometime you can get into a situation where 
the SHM is not intialized properly. Usually you will see an error like: 
TypeError: buffer is too small for requested array

To reset a SHM you can run the following code. Note: it will throw some garbage collector errors.
"""
# shm_names = ["wfs", "wfsRaw", "wfc", "wfc2D", "signal", "signal2D", "psfShort", "psfLong"] #list of SHMs to reset
# clear_shms(shm_names)

In [ ]:
# Load the configuration file
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        conf = yaml.safe_load(file)
    return conf

#Now we can read our YAML config file 
conf = read_yaml_file("simple_OOPAO_config.yaml")

#And separate it into sections for each of our AO loop components
confLoop = conf["loop"]
confWFS = conf["wfs"]
confWFC = conf["wfc"]
confPSF = conf["psf"]
confSlopes = conf["slopes"]

print(confLoop)
print(confWFS)
print(confWFC)
print(confPSF)
print(confSlopes)

In [ ]:
from simpleParamFile import *
param = initializeParameterFile()

In [ ]:
"""
Create the OOPAO simulation interface object 
Running this cell will initialize the dm, wfs, psf, and slopes objects, 
but will not start their real time computations. This inialization includes
the creation of the Shared Memory Objects, and the simulation inialization.
"""
sim = OOPAOInterface(conf=conf, param=param)
wfs, dm, psf = sim.get_hardware()

plt.imshow(wfs.wfs.cam.frame)
plt.show()

plt.imshow(dm.layout)
plt.show()

print(f"NUM VALID ACT: {np.sum(dm.layout)}")

In [ ]:
"""
It's important to set the full basis and number of possible modes before
initializing the loop object. Here I define a KL basis for the system
"""
from OOPAO.calibration.compute_KL_modal_basis import compute_KL_basis

NUM_MODES = 21

M2C_KL = compute_KL_basis(sim.tel, sim.atm, sim.dm)
dm.setM2C(M2C_KL[:,:NUM_MODES])

"""
"""
slopes = SlopesProcess(conf=conf)

""" 
"""
#Initialize our AO loop object
loop = Loop(conf=conf)

In [ ]:
### Setup the RTGym config ###

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = LoopGymInterface

my_config["interface_args"] = (loop,)
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 2.0
my_config["ep_max_length"] = 100
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False

In [ ]:
### Make the Gym env ###

env = gymnasium.make("real-time-gym-v1", config=my_config)

In [ ]:
"""
Start the processes. Here the real-time computations selected in
the config will begin.
"""
dm.start()
dm.flatten()

wfs.start()
slopes.start()

#Take new reference slopes while dm is flat.
# time.sleep(1)
# slopes.takeRefSlopes()

print(sim.dm.OPD.shape)
psf.start()

In [ ]:
sim.removeAtmosphere()

In [ ]:
#Remove the atmosphere from the simulation
sim.removeAtmosphere()

loop.pokeAmp = 1e-7

#Compute the IM, blocking
loop.computeIM()

#Add the atmosphere back to the simulation
sim.addAtmosphere()

In [ ]:
loop.plotIM()

In [ ]:
dm.flatten()
time.sleep(1e-2)
loop.setGain(0.3)
loop.start()
time.sleep(20)
loop.stop()
dm.flatten()

In [ ]:
dm.push(10, 1e-6)

In [ ]:
# loop.saveIM("simpleIM.npy")